In [1]:
from transformers.utils import logging

In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device("cpu")

In [3]:
# Загружаем модель ruGPT от сбера
model_name_or_path = "../huggingface/hub/rugpt3large_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(DEVICE)

In [4]:
!ls "../huggingface/hub/rugpt3large_based_on_gpt2"

README.md               merges.txt              tokenizer_config.json
config.json             pytorch_model.bin       vocab.json
flax_model.msgpack      special_tokens_map.json


In [5]:
# prompt engineering 
text = "Вопрос: 'Сколько будет 2+2?' \n Ответ: " # работает
# text = " Вопрос: 'Сколько будет 3+3?' \n Ответ: 6 . \n Вопрос: 'Сколько будет 1+9?' \n Ответ: 10 . \n Вопрос: 'Сколько будет 4+2?' \n Ответ:" # ValueError
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
out = model.generate(input_ids, do_sample=False) 

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

/opt/homebrew/lib/python3.11/site-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(



Вопрос: 'Сколько будет 2+2?' 
 Ответ: `2+2=4


1. %pip install transformers
2. загружаем Сберовскую модель.
3. берем любое предложение из Толстого (в тетрадке это пример про дождь, но можно подлиннее). 
4. пытаемся генерировать текст.
5. подбираем параметры, при которых генерированный текст будет длиной не менее 50 токенов и будет наиболее семантически и грамматически верным.
6. сдаем мне текст с описанием параметров.

In [10]:
# возьмем какой-нибудь текст
text = 'За окном дождь. Холодный и противный. Хочется'
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)

In [11]:
# если повторить запуск результат не изменится
# но тут уже нет зацикливания потому что модель смотрит дальше чем два токена в прошлое
out = model.generate(input_ids, do_sample=False, max_length=50)


generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


За окном дождь. Холодный и противный. Хочется плакать.

— Я не могу, — говорит она. — Я не могу.

— Почему?

— Потому что я не могу.




In [12]:
out = model.generate(input_ids, do_sample=True,  
                     top_k=0,  # про это параметр ниже
                     max_length=50)


generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


За окном дождь. Холодный и противный. Хочется уехать куда-нибудь.

В голове сложная, запутанная, путающаяся и противоречивая, но в тоже время такая ясная и продуманная, программа. Варианты сейчас, а завтра


## top_k
Этот параметр ограничивает количество слов, из которых мы семплируем. 10 - означает, что мы выбирает только из 10 самых вероятных слов. В ячейке выше мы поставили top_k = 0, потому что по умолчанию он стоит 50, а нам нужно было попробовать без него.

Чем больше top_k тем более случайный результат мы получим, но слишком низкий top_k может плохо сказаться на разнообразности.

In [13]:
for top_k in [1,3,10, 30, 100]:

    out = model.generate(input_ids, do_sample=True,  
                     top_k=top_k,  # про это параметр ниже
                     max_length=50)


    generated_text = list(map(tokenizer.decode, out))[0]
    print("### text with top_k - ", top_k)
    print(generated_text)
    print()

### text with top_k -  1
За окном дождь. Холодный и противный. Хочется плакать.

— Я не могу, — говорит она. — Я не могу.

— Почему?

— Потому что я не могу.



### text with top_k -  3
За окном дождь. Холодный и противный. Хочется закрыть окно и не открывать. Но, если закрыть окно, то можно простудиться, а это не очень приятно.

Я не знаю, как можно было жить в такой стране.



### text with top_k -  10
За окном дождь. Холодный и противный. Хочется спрятаться под теплое одеяло и никуда больше не выходить.<s>
Друзья, кто подскажет, что за фильм, и как называется? +++
"Круто"<s>
Какую

### text with top_k -  30
За окном дождь. Холодный и противный. Хочется спать. А у двери сидит незнакомый мужчина с большими грустными глазами за которыми блестят слёзы. Он говорит, что его зовут Саша, он музыкант, и играет в джаз-банде, которая

### text with top_k -  100
За окном дождь. Холодный и противный. Хочется плюнуть ему в лицо. И чтобы потом при этом никого не ждало раскаяние за содеянное. И что

# Сэмплирование с Температурой
Еще случайность можно контролировать с помощью параметра, который называется температура. Температура изменяет распределение - при низком значении температуры вероятности переносятся от низких значений к высоким (распределение заостряется), а при высоком - вероятности переносятся от высоких значений к низким (распределение сглаживается).
https://camo.githubusercontent.com/6c20c0e34ce075adbf6bc0fd757b998e38319f819c5bbdca6e6a2b50b7f7b405/68747470733a2f2f6d69726f2e6d656469756d2e636f6d2f76322f726573697a653a6669743a313430302f312a794a55772d7a6b70546671645456654f4f374f4b6e512e706e67![image.png](attachment:image.png)

Нулевая температура означает, что мы на каждом шаге просто выбираем по argmax(), а очень большая температура будет приводить к полному рандому. Под конкретную задачу температуру нужно подбирать отдельно, можно начать с 0 и постепенно увеличивать, смотря на получаемое разнобразие.

(температурой это называется потому что формула взята из физических уравнений, где этот параметр действительно отвечает за температуру)

In [14]:
for temp in [0.001, 0.1, 0.2, 0.5, 0.7, 1., 1.5, 2., 3., 4., 5.]:

    out = model.generate(input_ids, do_sample=True,  
                     top_k=0, 
                     temperature=temp,
                     max_length=50)


    generated_text = list(map(tokenizer.decode, out))[0]
    print("### text with temp - ", temp)
    print(generated_text)
    print()

### text with temp -  0.001
За окном дождь. Холодный и противный. Хочется плакать.

— Я не могу, — говорит она. — Я не могу.

— Почему?

— Потому что я не могу.



### text with temp -  0.1
За окном дождь. Холодный и противный. Хочется плакать.

– Ты не плачь, – говорит мама. – Ты не плачь.

– Я не плачу, – говорю я. – Я

### text with temp -  0.2
За окном дождь. Холодный и противный. Хочется спрятаться под одеяло и закрыть глаза.

— Ты не спишь? — спрашивает мама.

— Нет, — отвечаю я.

— А я вот

### text with temp -  0.5
За окном дождь. Холодный и противный. Хочется плакать.

Я не знаю, почему я плачу.

Я не знаю, почему я плачу.

Я не знаю, почему я плачу.

Я не знаю, почему

### text with temp -  0.7
За окном дождь. Холодный и противный. Хочется под одеяло и не выходить из дома.<s>
Что делать, если я в депрессии? Как справиться с ней?
Отдыхать.<s>
Что делать, если я в депрессии? Как

### text with temp -  1.0
За окном дождь. Холодный и противный. Хочется в теплую, нежную постель. Д

# Beam Search
У подходов выше есть недостаток - на каждом шаге выбирается только 1 слово и этот выбор нельзя изменить, поэтому 1 неверно выбраное слово можно испортить весь текст и это сложно проконтролировать температурой и топ-к.

С этим может помочь beam-search (поиск пучком). Напомню, что в нем по сути происходит одновременная генерация нескольких текстов параллельно и в конце выбирается текст с наибольшей общей вероятностью. Генерировать все возможные варианты невозможно технически (потому что количество вариаций растет очень быстро), поэтому в beam search варианты отсеиваются на каждом шаге так, чтобы количество текущих вариантов было не больше N. Этот параметр N (размер пучка, beam size) настраивается, но поставить его слишком большим не получится, т.к. опять же будет слишком много комбинаций и это увеличит время генерации.

In [15]:
# beam search уже реализован в hg поэтому нужно только задать параметр num_beams
out = model.generate(input_ids, do_sample=True, num_beams=5, top_k=0, max_length=60)

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text.replace('<s>', ' '))


За окном дождь. Холодный и противный. Хочется в тепло, в уют, где тепло и уютно. 
А у вас бывает такое настроение, когда хочется, чтобы все было по-другому?
бывает. 
А у вас бывает такое настроение, когда хочется, чтобы все было


In [16]:
# возьмем модель поменьше, так как дообучение это обновление весов
# model_name_or_path = "sberbank-ai/rugpt3small_based_on_gpt2"
model_name_or_path = "../huggingface/hub/models--sberbank-ai--rugpt3small_based_on_gpt2/snapshots/a9307e696cd3c5b7f953ff4cb19d76a4d81821d5"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path, use_cache=False).to(DEVICE)

In [6]:
text = """
«Мой дядя самых честных правил,
Когда не в шутку занемог,
Он уважать себя заставил
И лучше выдумать не мог.
Его пример другим наука;
Но, боже мой, какая скука
С больным сидеть и день и ночь,
Не отходя ни шагу прочь!
Какое низкое коварство
Полуживого забавлять,
Ему подушки поправлять,
Печально подносить лекарство,
Вздыхать и думать про себя:
Когда же черт возьмет тебя!»

Так думал молодой повеса,
Летя в пыли на почтовых,
Всевышней волею Зевеса
Наследник всех своих родных. —
Друзья Людмилы и Руслана!
С героем моего романа
Без предисловий, сей же час
Позвольте познакомить вас:
Онегин, добрый мой приятель,
Родился на брегах Невы,
Где, может быть, родились вы
Или блистали, мой читатель;
Там некогда гулял и я:
Но вреден север для меня.

Служив отлично-благородно,
Долгами жил его отец,
Давал три бала ежегодно
И промотался наконец.
Судьба Евгения хранила:
Сперва Madame за ним ходила,
Потом Monsieur ее сменил;
Ребенок был резов, но мил.
Monsieur l’Abbe€, француз убогой,
Чтоб не измучилось дитя,
Учил его всему шутя,
Не докучал моралью строгой,
Слегка за шалости бранил
И в Летний сад гулять водил.

Когда же юности мятежной
Пришла Евгению пора,
Пора надежд и грусти нежной,
Monsieur прогнали со двора.
Вот мой Онегин на свободе;
Острижен по последней моде;
Как dandy лондонский одет —
И наконец увидел свет.
Он по-французски совершенно
Мог изъясняться и писал;
Легко мазурку танцевал
И кланялся непринужденно;
Чего ж вам больше? Свет решил,
Что он умен и очень мил.

Мы все учились понемногу
Чему-нибудь и как-нибудь,
Так воспитаньем, слава богу,
У нас немудрено блеснуть.
Онегин был, по мненью многих
(Судей решительных и строгих),
Ученый малый, но педант.
Имел он счастливый талант
Без принужденья в разговоре
Коснуться до всего слегка,
С ученым видом знатока
Хранить молчанье в важном споре
И возбуждать улыбку дам
Огнем нежданных эпиграмм.

Латынь из моды вышла ныне:
Так, если правду вам сказать,
Он знал довольно по-латыни,
Чтоб эпиграфы разбирать,
Потолковать об Ювенале,
В конце письма поставить vale,
Да помнил, хоть не без греха,
Из Энеиды два стиха.
Он рыться не имел охоты
В хронологической пыли
Бытописания земли;
Но дней минувших анекдоты,
От Ромула до наших дней,
Хранил он в памяти своей.

Высокой страсти не имея
Для звуков жизни не щадить,
Не мог он ямба от хорея,
Как мы ни бились, отличить.
Бранил Гомера, Феокрита;
Зато читал Адама Смита
И был глубокий эконом,
То есть умел судить о том,
Как государство богатеет,
И чем живет, и почему
Не нужно золота ему,
Когда простой продукт имеет.
Отец понять его не мог
И земли отдавал в залог.

Всего, что знал еще Евгений,
Пересказать мне недосуг;
Но в чем он истинный был гений,
Что знал он тверже всех наук,
Что было для него измлада
И труд, и мука, и отрада,
Что занимало целый день
Его тоскующую лень, —
Была наука страсти нежной,
Которую воспел Назон,
За что страдальцем кончил он
Свой век блестящий и мятежный
В Молдавии, в глуши степей,
Вдали Италии своей.


Как рано мог он лицемерить,
Таить надежду, ревновать,
Разуверять, заставить верить,
Казаться мрачным, изнывать,
Являться гордым и послушным,
Внимательным иль равнодушным!
Как томно был он молчалив,
Как пламенно красноречив,
В сердечных письмах как небрежен!
Одним дыша, одно любя,
Как он умел забыть себя!
Как взор его был быстр и нежен,
Стыдлив и дерзок, а порой
Блистал послушною слезой!

Как он умел казаться новым,
Шутя невинность изумлять,
Пугать отчаяньем готовым,
Приятной лестью забавлять,
Ловить минуту умиленья,
Невинных лет предубежденья
Умом и страстью побеждать,
Невольной ласки ожидать,
Молить и требовать признанья,
Подслушать сердца первый звук,
Преследовать любовь и вдруг
Добиться тайного свиданья…
И после ей наедине
Давать уроки в тишине!


Как рано мог уж он тревожить
Сердца кокеток записных!
Когда ж хотелось уничтожить
Ему соперников своих,
Как он язвительно злословил!
Какие сети им готовил!
Но вы, блаженные мужья,
С ним оставались вы друзья:
Его ласкал супруг лукавый,
Фобласа давний ученик,
И недоверчивый старик,
И рогоносец величавый,
Всегда довольный сам собой,
Своим обедом и женой.


"""

In [7]:
from transformers import TextDataset, DataCollatorForLanguageModeling

# Сохраним обучающие данные в .txt файл 
train_path = 'train_dataset.txt'
with open(train_path, "w") as f:
    f.write(text)

# Создание датасета
train_dataset = TextDataset( tokenizer=tokenizer,file_path=train_path,block_size=64, 
                            overwrite_cache=True)
  
# специальный класс который будет подавать в модель данные в нужном ей виде
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

/opt/homebrew/lib/python3.11/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [8]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments( 
    output_dir= "./finetuned",
    overwrite_output_dir=True,
    num_train_epochs=100, 
    per_device_train_batch_size=32, 
    per_device_eval_batch_size=32,  
    gradient_accumulation_steps=16, 
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None) # Optimizer and lr scheduler
)

In [ ]:
trainer.train()

Step,Training Loss


In [ ]:
import torch
torch.mps

In [ ]:
DEVICE = torch.device("mps")

In [ ]:
text = "Дождь идет "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        temperature=0.8,
                        top_k=50,
                        max_length=300,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

In [ ]:
torch.hub.get_dir()